In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import ipywidgets as widgets
from matplotlib.cm import viridis
from matplotlib.colors import to_hex

import gmaps
import gmaps.geojson_geometries
import gmaps.datasets
import os
import json
import requests
import pprint as pp

import csv

# Import API key
from config import g_key

In [103]:
# column_list = list(data.columns)
# column_list

In [107]:
# # Grab useful columns
# NYC_df = data[['id','name','neighbourhood','latitude',
#                            'longitude','price','number_of_reviews',
#                           'review_scores_rating','review_scores_location','reviews_per_month']]
# NYC_df

In [108]:
# # Show DataFrame
# NYC_df.dropna()
# NYC_df.to_csv(r'FixedNYClisting.csv')

In [110]:
NYC_df = pd.read_csv('FixedNYClisting.csv')
NYC_df.head()

,Unnamed: 0,id,name,neighbourhood,latitude,longitude,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month
0,0,2595,Skylit Midtown Castle,Midtown,40.75362,-73.98377,$225.00,46,95.0,10.0,0.39
1,2,3831,Cozy Entire Floor of Brownstone,Brooklyn,40.68514,-73.95976,$89.00,274,90.0,10.0,4.64
2,3,5022,Entire Apt: Spacious Studio/Loft by central park,East Harlem,40.79851,-73.94399,$80.00,9,93.0,9.0,0.10
3,4,5099,Large Cozy 1 BR Apartment In Midtown East,Midtown East,40.74767,-73.97500,$200.00,75,89.0,9.0,0.60
4,5,5121,BlissArtsSpace!,Brooklyn,40.68688,-73.95596,$60.00,49,90.0,9.0,0.39


In [11]:
# Grab ratings
neighborhood_ratings = NYC_df[['neighbourhood','review_scores_location']]

In [17]:
neighborhood_ratings.dropna()
NYC_loc_scores_df = neighborhood_ratings.groupby('neighbourhood').mean()
NYC_loc_scores_df.head()

,review_scores_location
neighbourhood,
Allerton,9.636364
Alphabet City,9.695067
Annadale,10.000000
Arden Heights,10.000000
Arrochar,8.000000


In [74]:
NYC_loc_scores_df.to_csv(r'nyc_loc_scores.csv')

In [87]:
csv_data = pd.read_csv('nyc_loc_scores.csv')
csv_data

,neighbourhood,review_scores_location
0,Allerton,9.636364
1,Alphabet City,9.695067
2,Annadale,10.000000
3,Arden Heights,10.000000
4,Arrochar,8.000000
5,Astoria,9.624769
6,Bath Beach,9.714286
7,Battery Park City,10.000000
8,Bay Ridge,9.721649
9,Baychester,9.296296


In [33]:
# Authenticate gmaps
gmaps.configure(api_key=g_key)

# Load in NYC GeoJSON
with open('resources/custom-pedia-cities-nyc-Mar2018.json') as f:
    neighborhoods_geojson = json.load(f)

# Turn Neighborhoods Rating DataFrame into Dictionary
avg_location_dict = NYC_loc_scores_df.to_dict()
neighborhood2gini = avg_location_dict['review_scores_location']
min_gini = min(neighborhood2gini.values())
max_gini = max(neighborhood2gini.values())
gini_range = max_gini - min_gini

def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = viridis(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

# Calculate a color for each GeoJSON feature
colors = []
unique_colors = {}
for feature in neighborhoods_geojson['features']:
    neighborhood_name = feature['properties']['neighborhood']
    try:
        gini = neighborhood2gini[neighborhood_name]
        color = calculate_color(gini)
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.9)
    colors.append(color)
    if color not in unique_colors.keys():
        unique_colors[color] = gini



# Set Map
fig = gmaps.figure()
gini_layer = gmaps.geojson_layer(
    neighborhoods_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.9)
fig.add_layer(gini_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [86]:
# Authenticate gmaps
gmaps.configure(api_key=g_key)

nyc_coordinates = [(40.7128, 74.0060),(37.7749, 122.4194)]

# Load in NYC GeoJSON
with open('resources/custom-pedia-cities-nyc-Mar2018.json') as f:
    neighborhoods_geojson = json.load(f)

# Turn Neighborhoods Rating DataFrame into Dictionary
avg_location_dict = NYC_loc_scores_df.to_dict()
neighborhood2gini = avg_location_dict['review_scores_location']
min_gini = min(neighborhood2gini.values())
max_gini = max(neighborhood2gini.values())
gini_range = max_gini - min_gini

def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = viridis(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

# Calculate a color for each GeoJSON feature
colors = []
unique_colors = {}
legend_gini = [10.0, 9.5, 9.0, 8.5, 8.0, 7.5, 0]
legend_entries = []
#add black for null values
legend_entries.append('<span style="color: #000000; font-size: 2em; vertical-align: bottom;">&#9679;</span> 0.0')
# Some CSS hackery to draw a circle of that color next to the year
legend_template = '<span style="color: {color}; font-size: 2em; vertical-align: bottom;">&#9679;</span> {value}'

for feature in neighborhoods_geojson['features']:
    neighborhood_name = feature['properties']['neighborhood']
    try:
        gini = neighborhood2gini[neighborhood_name]
        color = calculate_color(gini)
        if gini in legend_gini:
            if color not in unique_colors.keys():
                unique_colors[color] = gini
                legend_entries.append(legend_template.format(color=color, value=gini))
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.9)
    colors.append(color)


# fix null value            
unique_colors[(0, 0, 0, 0.9)] = 0


def draw_map_with_legend():
    figure = gmaps.figure()
    

    gini_layer = gmaps.geojson_layer(
    neighborhoods_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.9)
    figure.add_layer(gini_layer)
    
    
    heatmap = gmaps.heatmap_layer(neighborhoods_geojson)
    heatmap.max_intensity = 100
    heatmap.point_radius = 100
    heatmap.gradient = ['black', 'white']

    
    legend = widgets.HBox(
        [widgets.HTML(legend_entry) for legend_entry in legend_entries],
        layout=widgets.Layout(width='100%', justify_content='space-between')
    )
    return widgets.VBox([figure, legend])

draw_map_with_legend()

ValueError: too many values to unpack (expected 2)